# 🧠 O que é ETL?

ETL é a sigla para Extract, Transform, Load, que em português significa Extrair, Transformar e Carregar. Esse é um processo fundamental em projetos de integração de dados e engenharia de dados, utilizado para mover e preparar dados de diferentes fontes para uso em análises, relatórios ou aplicações.


Aqui estou criando um processo de ETL que:

1. Extrai dados do Bitcoin usando uma API (como a da Binance).

2. Transforma os dados se necessário (por exemplo, converte timestamps, filtra colunas, essa parte ja sendo feita separadamente). 

3. Carrega esses dados em um banco de dados SQL Server (BitCointETL).

Um ponto importante é que o banco de dados utlizados aqui foi criado através do AWS RDS, também não irei me aprofundar nesse tópico do que é o RDS ou como se cria

In [7]:
#importações

import pandas as pd
import pyodbc
from datetime import datetime
from pegar_historico import pegar_historico
from dotenv import load_dotenv
import os
from datetime import date

In [43]:
#Para que eu nao colocasse minhas informações de segurança do banco direto no código, eu as coloquei em uma variavel de ambiente e as carrego utilizando
# a biblioteca load_dotenv, não irei me aprofunda aqui, mas basicamente, esse arquivo contem as informações de username. password, host, etc

#Carrega as informações
load_dotenv() 

#Cria a conexão com o banco de dados
connection_string = f"""
    DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={os.getenv('DB_HOST')},{os.getenv('DB_PORT')};
    DATABASE={os.getenv('DB_NAME')};
    UID={os.getenv('DB_USER')};
    PWD={os.getenv("DB_PASSWORD")};
    Encrypt=yes;
    TrustServerCertificate=yes;
"""

#Estabelece a conexao
conn = pyodbc.connect(connection_string)

#Faz a consulta no banco de dados
query = "SELECT * FROM bitcoin_data"
df = pd.read_sql(query, conn)
df

#Apesar de eu nao ter inserido dados por aqui, eu fiz uma inserção manual de dados diretamente pelo SQL, para algo pudesse ser exibido aqui

C:\Users\FilipeQueirozdeAbreu\AppData\Local\Temp\ipykernel_10780\3430104632.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,id,timestamp,open,high,low,close,volume
0,1,2025-01-01,583135.0,579057.0,591233.0,588693.0,72.14957


In [23]:
date.today().strftime("%b %Y")

'Jun 2025'

In [19]:
#Pegando os dados

par = "BTCBRL"
intervalo = "1d"
data_inicio = date.today().strftime("%#d %b %Y")

df_btcbrl = pegar_historico(
        par,
        intervalo,
        data_inicio
    )

#filtrado as colunas
df_btcbrl = df_btcbrl[
    ['timestamp','open','low','high','close','volume']
]

df_btcbrl.head(5)

,timestamp,open,low,high,close,volume
0,2025-06-05,592931.0,570500.0,596717.0,571805.0,163.25379


In [37]:
#Iterando as linhas em uma tupla

data_to_insert = [
    (
        row.timestamp, 
        row.open, 
        row.low, 
        row.high, 
        row.close, 
        row.volume
    )
    for _, row in df_btcbrl.iterrows()
]

data_to_insert

(Timestamp('2025-06-05 00:00:00'),
 592931.0,
 570500.0,
 596717.0,
 571805.0,
 163.25379)

In [45]:
conn = pyodbc.connect(connection_string)

#Construindo a query de inserção
insert_query = """
    INSERT INTO bitcoin_data VALUES (?, ?, ?, ?, ?, ?)
"""

cursor =  conn.cursor()
cursor.execute(insert_query, data_to_insert[0])
conn.commit()

In [47]:
#Verificando se a linha foi inserindo
df = pd.read_sql(query, conn)
df

C:\Users\FilipeQueirozdeAbreu\AppData\Local\Temp\ipykernel_10780\1061802609.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,id,timestamp,open,high,low,close,volume
0,1,2025-01-01,583135.0,579057.0,591233.0,588693.0,72.14957
1,2,2025-06-05,592931.0,570500.0,596717.0,571805.0,163.25379
